# TP Relation Extraction

## Introduction

Dans ce TP nous allons:

1. Découvrir la tache d’extraction de relation par réseaux neuronaux  (« neural relation extraction », NRE)
2.  Comprendre le fonctionnement d’un modèle d’extraction de relation avec un encodeur BERT
3. Découvrir la tache de reconnaissance d'entités nommées (« named-entity recognition », NER)
4. Coder une pipeline d’extraction de relation par réseaux neuronaux pour des jeux de données textuels

Avec les outils suivants:
1. La libraire [OpenNRE](https://github.com/thunlp/OpenNRE), basée sur Pytorch et HuggingFace’s Transformers, pour la tache d’extraction de relation par réseaux neuronaux
2. [HuggingFace’s Transformers](https://huggingface.co/transformers/) : une bibliothèque basée sur Pytorch pour le traitement automatique des langues et notamment les modèles neuronaux de type Transformer (comme BERT)
3. Google Colab, qui héberge ce *Jupyter Notebook*. Avant de commencer le TP, vous pouvez consulter des pages d'introductions [à Colab](https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb#scrollTo=YHI3vyhv5p85) et [aux Notebooks](https://realpython.com/jupyter-notebook-introduction/)


Contrairement au dernier TP, cette fois-ci nous n’aurons pas besoin d’utiliser une GPU car nous n’allons pas entraîner des nouveaux modèles ou faire de l’inférence sur des grands jeux de données: une CPU suffira. Nous pouvons vérifier si l’on est en train d’utiliser une CPU ou une GPU avec les lignes suivantes:

In [ ]:
import torch

if torch.cuda.is_available():
  print("GPU is available.")
  device = torch.cuda.current_device()
else:
  print("Will work on CPU.")

Nous avons besoin d’installer l’outil OpenNRE. Pour éviter de devoir ré-télécharger le répertoire  GitHub de OpenNRE à chaque fois qu’on ré-initialise le fichier Colab, il est convenant de monter notre répertoire Google Drive et y télécharger le répertoire OpenNRE de façon à l’avoir toujours disponible. Ainsi, à chaque fois que nous allons ré-initialiser le fichier Colab, il nous suffira de monter notre Goodle Drive pour avoir accès à la libraire OpenNRE.

> ATTENTION: modifiez *tp_path_in_drive* pour pointer sur le repertoire où vous avez placé le fichier tp_re.ipynb, vous allez télécharger OpenNRE dans le meme réépertoire. Si vous êtes sur votre machine locale, vous n'avez pas besoin de monter le Drive, mais juste de faire le clonage du réépertoire OpenNRE.

In [ ]:
import os
from google.colab import drive

tp_path_in_drive = '/content/gdrive/My Drive/tp_ensimag/tp_re'
opennre_path_in_drive = tp_path_in_drive + '/OpenNRE'

# mount Google Drive
drive.mount('/content/gdrive')

if not os.path.isdir(opennre_path_in_drive):
  # OpenNRE is not already present in Google Drive
  if not os.path.isdir(tp_path_in_drive):
    # make directory for the TP if necessary
    os.makedirs(tp_path_in_drive, exist_ok=True)
  # change directory to the TP directory
  os.chdir(tp_path_in_drive)
  # clone OpenNRE repo
  print("Cloning repo...")
  os.system('git clone https://github.com/thunlp/OpenNRE.git')
  print("...done!")
else:
  print("OpenNRE is already present in Google Drive under {0}".format(opennre_path_in_drive))

# Change current dir to OpenNRE
os.chdir(opennre_path_in_drive)

In [ ]:
# Update requirements requirements
!sed -i '/transformers==3.0.2/c\transformers==3.4.0' requirements.txt

Nous pouvons désormais continuer avec l’installation.

In [ ]:
!pip install -r requirements.txt
!python setup.py install

## Les modéles pour l'extraction de relation

À part offrir un cadre pour l’implémentation et l’entraînement de modelés pour l’extraction de relation, OpenNRE offre aussi des modèles déjà entraînés sur différents jeux de données, et donc capables de détecter différents types de relations entre entités. 

Ici, nous allons employer un modèle entraîné sur Wiki80 (dataset introduit par [le papier OpenNRE](https://www.aclweb.org/anthology/D19-3029/)), un jeu de données contenant des phrases collectées sur Wikipedia et Wikidata, ainsi que des rélations entre leurs entités. Si vous voulez en savoir plus sur Wiki80, vous pouvez le télécharger avec le script [download_wiki80.sh](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/benchmark/download_wiki80.sh) 

In [ ]:
import opennre
model = opennre.get_model('wiki80_bert_softmax')

Nous pouvons utiliser ce modèle pour calculer la relation entre un mot «tête» et un mot «queue» qui sont contenus dans un texte. Il suffit de passer au modèle le texte ainsi que la position de la tête et de la queue. Le modèle retournera la relation de la tête à l’égard de la queue ainsi que la probabilité qu’il associe à cette rélation. Par example, nous pouvons inféérer la relation entre **Áed Uaridnach** et **Máel Dúin mac Máele Fithrich** de la façon suivante:

In [ ]:
item = {'text': 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).', 'h': {'pos': (78, 91)}, 't': {'pos': (18, 46)}}
print(model.infer(item))

#### Exercice 1



Écrire une fonction `to_input_format(text, head, tail)` qui nous permet de trouver la position de deux mots (une tête et une queue) dans un texte, et qui retourne un dictionnaire contenant le texte et les deux positions suivant le format requis par la fonction `model.infer()`.

In [ ]:
import re

def to_input_format(text, head, tail):
  """
  Args:
    text: a string of text
    head: a string of text representing a word contained in text
    tail: a string of text representing a word contained in text but different from head
  Returns:
    A dictionary containing the text and the position of head and tail within it,
    following the input format required by a SoftmaxNN model.
  """
  # ....

In [ ]:
# Test your code with this snippet
text = 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).'
head = 'Áed Uaridnach'
tail = 'Máel Dúin mac Máele Fithrich'
test_item = to_input_format(text, head, tail)
try:
  assert model.infer(item) == model.infer(test_item)
  print("Good job!")
except AssertionError:
  print("Something is wrong with your function, try again!")

#### Exercice 2

Décrire en quelque ligne l’architecture du modèle qu’on vient de télécharger, ainsi que sa logique, en faisant de références à ce que vous avez vu dans le cours ainsi que ce que vous avez appris dans le TP BERT.

Aide: vous pouvez inspecter un modèle Pytorch avec `print(model)`. Pour mieux le comprendre, vous pouvez voir aussi son [code](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/opennre/model/softmax_nn.py#L5).

## Encodeur

#### Exercice 3

Maintenant, nous allons essayer de mieux comprendre le fonctionnement de ce modèle avec un focus sur son encodeur, qui est définit dans le fichier [bert_encoder.py](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/opennre/encoder/bert_encoder.py#L7). Ce qui est spécifique à la tache de NRE dans ce modèle n’est pas l’architecture, mais plutôt la façon dont la séquence en input est tokenisée. Avec l’objectif de bien comprendre comment cet encodeur gère son input, répondez en détail aux questions suivantes qui se référent à la méthode `tokenize`. Si vous utilisez du code pour vous aider à répondre (conseillé), veuillez le joindre à vos réponses textuelles.

1. Qu’est-ce qu’elle sont les variables `sent0`, `ent0`, `sent1`, `ent1`, `sent2` ?
2. Qu’est-ce que c’est `re_tokens` ?
3. Dans le _forward_, le `BERTEncoder` prend en entrée exclusivement la séquence textuelle qui a étée préalablement tokenisée. Comment est-il capable de distinguer la tête et la queue en sort de (apprendre à) prédire la relation entre les deux ?
4. Qu’est-ce que c’est le « Padding » et pourquoi est-il utile ?
5. Qu’est-ce que c’est l’ « Attention mask » et pourquoi est-elle utile ?
6. Quelle est la différence entre la classe `BERTEncoder` et la classe `BERTEntityEncoder`, définie dans le même fichier ?


Aide : vous pouvez accéder à la méthode `tokenize` pour la tester avec `model.sentence_encoder.tokenize(item)`

In [ ]:
# Write your code here if needed, and then write your detailed answers below.

## NRE Pipeline

Nous allons maintenant programmer une application qui prend en entrée une phrase et donne en sortie deux entités nommées dans la phrase ainsi que la relation entre eux.

Pour ce faire, nous avons besoin :

1. d’un système NER, qui reconnaît les entités nommées dans la phrase
2. d’un système NRE, comme celui que nous avons utilisé jusqu’à là

#### Exercice 4

En vous aident avec la documentation de HuggingFace, instanciez une [pipeline](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipeline#the-pipeline-abstraction) `ner_pipeline` pour la reconnaissance d'entités nommées, avec le modèle et le tokeniseur pré-entraînes [dbmdz/bert-large-cased-finetuned-conll03-english](https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).

Vous pouvez tester la bonne réussite de l’exercice avec le code ci-dessous.

**Indice** : la solution consiste en deux lignes de code : l'une pour importer la classe pipeline, l'autre pour instancier la bonne pipeline.

In [ ]:
# Write your code here

In [ ]:
# Apply your ner_pipeline to some sentences to see how it works,
# Then you can test your code with this snippet
text = 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).'
try:
  assert ner_pipeline(text) == [
                                {'entity': 'I-PER', 'index': 6, 'score': 0.982085645198822, 'word': 'M'},
                                {'entity': 'I-PER', 'index': 7, 'score': 0.8588606715202332, 'word': '##á'},
                                {'entity': 'I-PER', 'index': 8, 'score': 0.876600980758667, 'word': '##el'},
                                {'entity': 'I-PER', 'index': 9, 'score': 0.8375431299209595, 'word': 'D'},
                                {'entity': 'I-PER', 'index': 10, 'score': 0.4897184371948242, 'word': '##ú'},
                                {'entity': 'I-PER', 'index': 11, 'score': 0.9012535810470581, 'word': '##in'},
                                {'entity': 'I-PER', 'index': 12, 'score': 0.5821399688720703, 'word': 'mac'},
                                {'entity': 'I-PER', 'index': 13, 'score': 0.8884767889976501, 'word': 'M'},
                                {'entity': 'I-PER', 'index': 14, 'score': 0.7004088759422302, 'word': '##á'},
                                {'entity': 'I-PER', 'index': 15, 'score': 0.8791091442108154, 'word': '##ele'},
                                {'entity': 'I-PER', 'index': 16, 'score': 0.9720048308372498, 'word': 'Fi'},
                                {'entity': 'I-PER', 'index': 17, 'score': 0.6978078484535217, 'word': '##th'},
                                {'entity': 'I-PER', 'index': 18, 'score': 0.6970881223678589, 'word': '##rich'},
                                {'entity': 'I-PER', 'index': 26, 'score': 0.9203091859817505, 'word': 'Á'},
                                {'entity': 'I-PER', 'index': 27, 'score': 0.9416706562042236, 'word': '##ed'},
                                {'entity': 'I-PER', 'index': 28, 'score': 0.9702795147895813, 'word': 'U'},
                                {'entity': 'I-PER', 'index': 29, 'score': 0.8428964614868164, 'word': '##ari'},
                                {'entity': 'I-PER', 'index': 31, 'score': 0.707791805267334, 'word': '##ach'}
                                ]
  print("Good job!")
except AssertionError:
  print("Something might be wrong with your pipeline.") 

#### Exercice 5


Nous pouvons finalement développer une pipeline NRE reposante sur notre modèle de NRE et notre pipeline NER. 

Écrivez ci-dessous une classe `NREPipeline` équipée (entre autres) d'une méthode `__call__(self, text)` qui prend un texte en entrée et effectue les opérations suivantes :

- elle reconnaît les entités dans le texte
    - retenir seulement les deux entités auxquelles la pipeline NER associe la probabilité la plus élevée, écarter les autres (si presentes)
    - si la pipeline NER ne reconnaît qu’une seule entités dans le texte, `__call__(self, text)` retourne None (voir le test en bas) car il n’y a aucune relation à prédire
- elle donne en sortie une liste en format `[e1, e2, rel, p]` où :
    - `e1` est la première entité reconnue dans le texte (entre les deux plus probables, la première qui apparaître dans le texte)
    - `e2` est la deuxième entité  reconnue dans le texte (entre les deux plus probables, la deuxième qui apparaître dans le texte)
    - `rel` est la relation qu’il y a entre `e1` et `e2`
    - `p` est la probabilité associée à la relation `rel` par le modèle de NRE

Pour vérifier la bonne qualité de votre classe `NREPipeline`, utilisez l’extrait de code ci-dessous. Le résultat devrait ressabler à celui-ci :

````
Sentence 0: He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).
System out:  ['Máel Dúin mac Máele Fithrich', 'Áed Uari', 'father', 0.9923498034477234]
------------------------------
Sentence 1: He was the son of Máel Dúin
System out:  None
------------------------------
Sentence 2: Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .
System out:  ['Iwami Ginzan', 'World Heritage Site', 'heritage designation', 0.9991846680641174]
------------------------------
Sentence 3: It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .
System out:  None
------------------------------
Sentence 4: Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church .
System out:  ['Earleville', "St . Stephen ' s Episcopal Church", 'location', 0.9127373099327087]
------------------------------
````

In [ ]:
# Write your pipeline in this cell. The fun begins here, because we ar ecoding a whole Python class here !

class NREPipeline(object):
  def __init__(self, ner_pipeline, nre_model):
    # ...

  # ...

  def __call__(self, text):
   # ...

In [ ]:
# Test your code with this snippet

sequences = [
             'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).', # Easy sentence
             'He was the son of Máel Dúin', # There is only one entity in this sentence, therefore our pipeline should return None
             'Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .', # Ōda is tokenized by the NER tokenizer as a "[UNK]" and it is detected as an entitity. For simplicity, you can discard [UNK] entities as if they were not detected.
             'It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .', # the NER system can not recognise any entity here, therefore the pipeline should return None
             "Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church ." # This sentence is not trivial because the 's has to be managed properly

]

nre = NREPipeline(ner_pipeline, model)
for n, sequence in enumerate(sequences):
  out = nre(sequence)
  print("Sentence {0}: {1}".format(n, sequence))
  print("System out: ", out)
  print('------------------------------')

## Application de le pipeline

#### Exercice 6

1. Appliquez la pipeline NRE aux phrases contenues dans le fichier _sentences.txt_, qui ont étées extraites à partir de Wikipedia et Wikidata.

2. Donnez un avis qualitatif sur sa performance, les problèmes rencontrés ainsi que des idées pour améliorations des résultats (e.g. entraînement sur des données différentes, modèle différent, etc.).

3. Appliquez la pipeline NRE aux premières 50 phrases contenues dans le fichier _Sentences_AllAgree.txt_ que vous avez utilisé pour  TP BERT, qui parlent d’événements financiers

4. Remarquez-vous des différences en terme de performances par rapport à la question 2 ? Pourquoi ? Commentez…

In [ ]:
# Write your code here and then write your detailed answers below. Do not hesitate to use more code cells if needed